<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/deleting_old_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [ ]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [ ]:
!pip install -q canopy

In [ ]:
import canopy
import logging
from typing import Optional, List
import nest_asyncio

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

### Authenticate

In [ ]:
authentication_data = canopy.prompt_for_authentication()
session = canopy.Session(authentication_data)

# Example: Deleting old studies

**We advise using extreme caution when running this tool.**

The following code will iterate through the studies in the given time range and delete them. It will take ownership of the study before deleting if necessary.

 - You can do a trial run by setting the `trial_run` parameter to true, in which case no studies will actually be modified.

 - If the study is older than the 29th of March 2019 then taking ownership will fail, as studies before this date are not able to have their owner changed. 

 - You can optionally specify a username, in which case only studies owned by that user will be fetched for deletion.

 - You will have a limited time to restore a deleted study on the platform if you delete it accidentally.


In [ ]:
def create_filter(
    from_timestamp: str,
    to_timestamp: str,
    owner_username: Optional[str]) -> canopy.openapi.ListFilter:

    conditions: List[canopy.openapi.ListFilterCondition] = []

    conditions.append(canopy.openapi.ListFilterCondition(
        source='metadata',
        name='creationDate',
        operator='greaterThan',
        value=from_timestamp))

    conditions.append(canopy.openapi.ListFilterCondition(
        source='metadata',
        name='creationDate',
        operator='lessThan',
        value=to_timestamp))

    if owner_username is not None:
        owner = tenant_users.get_by_username(owner_username)
        conditions.append(canopy.openapi.ListFilterCondition(
            source='metadata',
            name='userId',
            operator='equals',
            value=owner.user_id
        ))

    query = canopy.openapi.ListFilterGroup(
                operator='and',
                conditions=conditions)
    
    study_filter = canopy.openapi.ListFilter(
        items_per_page=10,
        order_by_property='creationDate',
        order_by_descending=False,
        query=query,
        include_if_has_parent_worksheet=False)
        
    return study_filter

async def delete_studies(
    trial_run: bool,
    from_timestamp: str,
    to_timestamp: str,
    owner_username: Optional[str] = None):

    if trial_run == False:
        print('WARNING: This will potentially delete studies from multiple users. Type "continue" to proceed:')
        prompt_result = input()
        if prompt_result !=  'continue':
            print(f'Stopping as user responsed with "{prompt_result}"."')
            return

    session.authentication.authenticate()
    tenant_id = session.authentication.tenant_id
    user_id = session.authentication.user_id

    tenant_users = session.tenant_users.get(tenant_id)
    study_api = canopy.openapi.StudyApi(session.async_client)

    study_filter = create_filter(from_timestamp, to_timestamp, owner_username)

    has_more_results = True
    continuation_token = None
    deleted_study_count = 0
    failed_study_count = 0

    while has_more_results:
        print('Loading next studies...')
        study_filter.continuation_token = continuation_token
        serializable_filter = canopy.SerializableValue(session, study_filter);
        studies = await study_api.study_get_studies(tenant_id, filter=serializable_filter.serialize())

        for study in studies.query_results.documents:
            creation_date = study.creation_date.strftime('%Y-%m-%d %H:%M:%S')
            owner = tenant_users.get_by_user_id(study.user_id)

            print(f'{owner.username} / {study.name} / {creation_date} / https://portal.canopysimulations.com/studies/{tenant_id}/{study.document_id}')

            if trial_run == False:
                try:
                    if study.user_id != user_id:
                        print('Taking ownership...')
                        await study_api.study_put_study_owner(
                            tenant_id,
                            study.document_id,
                            canopy.openapi.ConfigOwnerData(user_id=user_id))
                    
                    print('Deleting...')
                    await study_api.study_delete_study(
                        tenant_id,
                        study.document_id)
                    print('Deleted.')
                    deleted_study_count += 1
                except canopy.openapi.ApiException as error:
                    failed_study_count += 1
                    logging.error(error.body)

        has_more_results = studies.query_results.has_more_results
        continuation_token = studies.query_results.continuation_token
    
    print(f'Successfully deleted {deleted_study_count} studies.')
    print(f'Failed to delete {failed_study_count} studies.')

await delete_studies(
    trial_run=True,
    #owner_username='james.thurley',
    from_timestamp='2019-03-27T13:00:00Z',
    to_timestamp='2019-03-27T16:00:00Z')